# reaction-network (Demo Notebook): Networks

## Author: Matthew McDermott
## Last Updated: 11/16/21

**If you use this code in your work, please consider citing the following paper:**

McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. 
Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

In [1]:
import logging 

from pymatgen.ext.matproj import MPRester
from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from rxn_network.thermo.chempot_diagram import ChemicalPotentialDiagram

from rxn_network.costs.softplus import Softplus
from pymatgen.core.composition import Composition, Element
from rxn_network.entries.entry_set import GibbsEntrySet
from rxn_network.network.network import ReactionNetwork
from rxn_network.entries.nist import NISTReferenceEntry
from rxn_network.reactions.computed import ComputedReaction
from rxn_network.reactions.reaction_set import ReactionSet
from rxn_network.reactions.open import OpenComputedReaction
from rxn_network.network.entry import NetworkEntry, NetworkEntryType
from rxn_network.network.visualize import plot_network_on_graphistry, plot_network
from rxn_network.pathways.solver import PathwaySolver

import graphistry
import graph_tool.all as gt

%load_ext autoreload
%autoreload 2

In [2]:
with MPRester() as mpr:
    entries = mpr.get_entries_in_chemsys("Y-Mn-O-Li-Cl-C", inc_structure="final")

  0%|          | 0/848 [00:00<?, ?it/s]

In [3]:
entry_set = GibbsEntrySet.from_entries(entries, 900, include_barin_data=False).filter_by_stability(0.0)

In [4]:
be = BasicEnumerator()
cf = Softplus(900)

In [5]:
rn = ReactionNetwork(entry_set, [BasicEnumerator()], cf)

In [6]:
rn.build()

BasicEnumerator:   0%|          | 0/57 [00:00<?, ?it/s]

In [7]:
rn.set_precursors([entry_set.get_min_entry_by_formula("Li2CO3"), 
                   entry_set.get_min_entry_by_formula("Mn2O3"), 
                   entry_set.get_min_entry_by_formula("YCl3")])

In [8]:
paths = rn.find_pathways(["YMnO3","LiCl","CO2"], k=5)

PATHS to YMnO3 

--------------------------------------- 

Li2CO3 -> Li2O + CO2 (dG = 0.277 eV/atom) 
Li2O + 0.6667 YCl3 -> 0.3333 Y2O3 + 2 LiCl (dG = -0.297 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.048 eV/atom) 
Total Cost: 0.794 

6 Mn2O3 -> O2 + 4 Mn3O4 (dG = 0.043 eV/atom) 
O2 + 2 YCl3 -> 2 YClO + 2 Cl2 (dG = 0.011 eV/atom) 
1.5 YClO + 0.5 Mn2O3 -> YMnO3 + 0.5 YCl3 (dG = 0.015 eV/atom) 
Total Cost: 0.811 

6 Mn2O3 -> O2 + 4 Mn3O4 (dG = 0.043 eV/atom) 
O2 + 1.333 YCl3 -> 2 Cl2 + 0.6667 Y2O3 (dG = 0.078 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.048 eV/atom) 
Total Cost: 0.813 

Li2CO3 -> Li2O + CO2 (dG = 0.277 eV/atom) 
Li2O + YCl3 -> YClO + 2 LiCl (dG = -0.273 eV/atom) 
1.5 YClO + 0.5 Mn2O3 -> YMnO3 + 0.5 YCl3 (dG = 0.015 eV/atom) 
Total Cost: 0.813 

Mn2O3 -> MnO2 + MnO (dG = 0.097 eV/atom) 
0.6667 YCl3 + MnO -> MnCl2 + 0.3333 Y2O3 (dG = 0.051 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.048 eV/atom) 
Total Cost: 0.819 

PATHS to LiCl 

--------------------------------------- 

Li

In [9]:
ps = PathwaySolver(rn.entries, paths, Softplus(900)) # open_elem="O", chempot=0

In [10]:
product_entries = []
for i in ["YMnO3","LiCl","CO2"]:
    product_entries.append(entry_set.get_min_entry_by_formula(i))
    
net_rxn = ComputedReaction.balance(rn.precursors,product_entries)
net_rxn

YCl3 + 0.5 Mn2O3 + 1.5 Li2CO3 -> YMnO3 + 3 LiCl + 1.5 CO2

In [15]:
a = ps.solve(net_rxn, max_num_combos=5, intermediate_rxn_energy_cutoff=0.0, filter_interdependent=True)

BasicEnumerator:   0%|          | 0/36 [00:00<?, ?it/s]

[(GibbsComputedEntry | mp-1182086 | C4 O8 (CO2)
Gibbs Energy (900 K) = -16.5242, 1.0), (GibbsComputedEntry | mp-3054 | Li4 C2 O6 (Li2CO3)
Gibbs Energy (900 K) = -21.5423, -0.9999999999999996), (NISTReferenceEntry | Li2O
Gibbs Energy (900 K) = -4.9775, 1.0000000000000002)]
[(NISTReferenceEntry | Li2O
Gibbs Energy (900 K) = -4.9775, -1.0), (NISTReferenceEntry | LiCl
Gibbs Energy (900 K) = -3.4727, 1.9999999999999996), (GibbsComputedEntry | mp-2652 | Y16 O24 (Y2O3)
Gibbs Energy (900 K) = -139.2136, 0.33333333333333326), (GibbsComputedEntry | mp-27455 | Y2 Cl6 (YCl3)
Gibbs Energy (900 K) = -18.2567, -0.666666666666666)]
[(GibbsComputedEntry | mp-1172875 | Mn32 O48 (Mn2O3)
Gibbs Energy (900 K) = -121.0665, -0.9999999999999998), (GibbsComputedEntry | mp-2652 | Y16 O24 (Y2O3)
Gibbs Energy (900 K) = -139.2136, -1.0000000000000002), (GibbsComputedEntry | mp-19227 | Y2 Mn2 O6 (YMnO3)
Gibbs Energy (900 K) = -25.4484, 1.9999999999999991)]
[(GibbsComputedEntry | mp-1172875 | Mn32 O48 (Mn2O3)
Gibbs 

PathwaySolver:   0%|          | 0/1 [00:00<?, ?it/s]

PathwaySolver:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
for path in a:
    print(path)
    print("\n")

Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Li2O + 0.6667 YCl3 -> 2 LiCl + 0.3333 Y2O3 (dG = -0.297 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.048 eV/atom) 
Average Cost: 0.268


Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Li2O + YCl3 -> 2 LiCl + YClO (dG = -0.273 eV/atom) 
0.5 Li2CO3 + 0.5 Mn2O3 -> 0.5 CO2 + LiMnO2 (dG = 0.08 eV/atom) 
LiMnO2 + YClO -> LiCl + YMnO3 (dG = -0.066 eV/atom) 
Average Cost: 0.27


Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Li2O + 0.6667 YCl3 -> 2 LiCl + 0.3333 Y2O3 (dG = -0.297 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.048 eV/atom) 
Li2O + YCl3 -> 2 LiCl + YClO (dG = -0.273 eV/atom) 
Li2O + 2 YClO -> 2 LiCl + Y2O3 (dG = -0.136 eV/atom) 
Average Cost: 0.271


Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.048 eV/atom) 
Li2O + YCl3 -> 2 LiCl + YClO (dG = -0.273 eV/atom) 
Li2O + 2 YClO -> 2 LiCl + Y2O3 (dG = -0.136 eV/atom) 
Average Cost: 0.273


